In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from models import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
activation = hard_sigmoid
criterion = torch.nn.MSELoss(reduction='none').to(device)

In [5]:
architecture = [784, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T

beta = 1
lambda_h = 0.999
lambda_y = 0.999
epsilon = 0.1
one_over_epsilon = 1 / epsilon
lr = {'ff' : 0.05, 'fb': 0.05, 'lat': 1e-3}
neural_lr = 0.05
model = TwoLayerCorInfoMax(architecture = architecture, lambda_h = lambda_h, lambda_y = lambda_y, 
                           epsilon = epsilon, activation = activation)

In [6]:
trn_acc_list = []
tst_acc_list = []
neural_dynamic_iterations_free = 20
neural_dynamic_iterations_nudged = 4
# lambda_h = 0.01
# lambda_y = 0.01
# epsilon = 1
# one_over_epsilon = 1 / epsilon
n_epochs = 50
# lr = {'ff' : 1e-3, 'fb': 1e-3, 'lat': 1e-3}
# neural_lr = 0.25

for epoch_ in range(n_epochs):
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T

        h, y_hat = model.batch_step(  x, y_one_hot, lr, neural_lr, neural_dynamic_iterations_free, 
                                      neural_dynamic_iterations_nudged, beta, output_sparsity = True, STlambda_lr = 0.00001)

    trn_acc = evaluateCorInfoMax(model, train_loader, neural_lr, 20, device = 'cuda', printing = False)
    tst_acc = evaluateCorInfoMax(model, test_loader, neural_lr, 20, device = 'cuda', printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)

    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [00:38, 77.49it/s]
7it [00:00, 68.60it/s]

Epoch : 1, Train Accuracy : 0.8700833333333333, Test Accuracy : 0.8794


3000it [00:38, 76.95it/s]
7it [00:00, 68.56it/s]

Epoch : 2, Train Accuracy : 0.8944, Test Accuracy : 0.9014


3000it [00:38, 77.75it/s]
8it [00:00, 73.62it/s]

...

Epoch : 48, Train Accuracy : 0.09871666666666666, Test Accuracy : 0.098


3000it [00:39, 76.04it/s]
8it [00:00, 77.45it/s]

Epoch : 49, Train Accuracy : 0.09871666666666666, Test Accuracy : 0.098


3000it [00:38, 77.91it/s]


Epoch : 50, Train Accuracy : 0.09871666666666666, Test Accuracy : 0.098


In [7]:
x, y = next(iter(train_loader))
x, y = x.to(device), y.to(device)
x = x.view(x.size(0),-1).T
y_one_hot = F.one_hot(y, 10).to(device).T
x.size()

torch.Size([784, 20])

In [8]:
h, y_hat = model.init_neurons(x.size(1), device = model.device)

In [9]:
h.shape

torch.Size([500, 20])

In [10]:
y.shape

torch.Size([20])

In [11]:
y_hat.shape

torch.Size([10, 20])

In [12]:
mbs = x.size(1)
STLAMBD = torch.zeros(1, mbs).to(model.device)

In [13]:
STLAMBD.shape

torch.Size([1, 20])

In [14]:
y_hat - STLAMBD

tensor([[ 0.1730, -0.8634,  0.6301, -1.0988,  0.9705,  0.1022, -0.9083, -0.2224,
         -1.1432,  0.1936, -0.4580,  1.4503,  0.2053,  1.9611,  2.1635,  0.5857,
         -1.2036, -0.3588,  1.8396, -1.1425],
        [ 0.2872, -1.1663,  1.3257,  0.4881, -0.0129,  0.1628,  2.0088,  0.3743,
          1.1896,  1.5644, -0.7767, -0.9679,  1.0421, -0.5070, -0.2747, -0.1750,
          1.8694,  0.0405, -1.3239,  0.0185],
        [ 0.1581,  0.1012,  0.7867, -0.4431,  0.6253,  0.0394, -0.6650,  1.2012,
          1.6767, -0.1975, -0.7631,  1.4566, -0.8213,  0.0490,  1.8932,  0.9232,
         -0.5656, -0.0114,  0.5854, -0.1215],
        [-0.1072, -0.2964,  0.4336,  1.0648, -0.4904, -2.4438, -0.3226,  0.0515,
          0.4592,  1.3150,  0.5700,  2.4130, -0.7307,  0.5552,  0.5461,  0.3059,
         -0.5799,  1.2142,  1.0255,  0.7172],
        [ 0.6512,  0.9543, -0.6445,  0.4787, -0.4106,  2.4511, -1.9784, -0.5561,
         -0.4335,  1.9568, -0.8228,  2.0941,  0.2480, -0.8179,  0.2607, -0.9639,
      

In [15]:
torch.sum(y_hat, 0).view(1, -1).shape

torch.Size([1, 20])